In [10]:
import pandas as pd
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Read JSON file into DataFrame
df = pd.read_json('Couvee Seturan.json')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         250 non-null    object 
 1   google_id    250 non-null    object 
 2   place_id     250 non-null    object 
 3   reviews      250 non-null    int64  
 4   rating       250 non-null    float64
 5   review_id    250 non-null    object 
 6   review_text  250 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 13.8+ KB


In [17]:
df = df.drop(columns= ['name', 'google_id', 'place_id', 'reviews', 'rating', 'review_id'])
df

,review_text
0,Couvee Seturan adalah salah satu tempat nongkr...
1,"couvee dah kaya rumahh, tempatnya nyaman, fasi..."
2,Aku gak tau gimana lagi mengungkapkan rasa cin...
3,Kopi dengan cita rasa terbaik dan didukung sua...
4,Couvee Seturan dikenal sebagai cabang yang ram...
...,...
245,"Aircon too low, kinda freezing inside. Wear wa..."
246,The price lil bit pricey. Not so many choices ...
247,I am so in love with red velvet latte. Best in...
248,Thanks for excellent service 👍 barista ok ob ok


In [18]:
# Hapus baris yang kosong
df = df.dropna(subset=['review_text'])

# Reset indeks setelah penghapusan
df.reset_index(drop=True, inplace=True)


In [19]:
# Hapus baris duplikat
df = df.drop_duplicates(subset=['review_text'])


In [20]:
df['cleaned_text'] = df['review_text'].str.lower()


In [21]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Hanya huruf dan spasi
    text = re.sub(r'\s+', ' ', text)  # Menghapus spasi berlebih
    return text.strip()

df['cleaned_text'] = df['cleaned_text'].apply(clean_text)


In [26]:
import nltk
nltk.download('punkt_tab')  # Unduh tokenizer yang diperlukan

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [27]:
df['tokens'] = df['cleaned_text'].apply(word_tokenize)

In [28]:
# Stopwords bahasa Indonesia dan Inggris
stop_words = set(stopwords.words('indonesian')).union(set(stopwords.words('english')))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

df['tokens'] = df['tokens'].apply(remove_stopwords)


In [31]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming
def stem_words(tokens):
    return [stemmer.stem(word) for word in tokens]

# Terapkan stemming pada kolom tokens
df['stemmed_tokens'] = df['tokens'].apply(stem_words)
print(df.head())


                                         review_text  \
0  Couvee Seturan adalah salah satu tempat nongkr...   
1  couvee dah kaya rumahh, tempatnya nyaman, fasi...   
2  Aku gak tau gimana lagi mengungkapkan rasa cin...   
3  Kopi dengan cita rasa terbaik dan didukung sua...   
4  Couvee Seturan dikenal sebagai cabang yang ram...   

                                        cleaned_text  \
0  couvee seturan adalah salah satu tempat nongkr...   
1  couvee dah kaya rumahh tempatnya nyaman fasili...   
2  aku gak tau gimana lagi mengungkapkan rasa cin...   
3  kopi dengan cita rasa terbaik dan didukung sua...   
4  couvee seturan dikenal sebagai cabang yang ram...   

                                              tokens  \
0  [couvee, seturan, salah, nongkrong, favorit, j...   
1  [couvee, dah, kaya, rumahh, tempatnya, nyaman,...   
2  [gak, tau, gimana, cintaku, kopi, minggu, kesi...   
3  [kopi, cita, terbaik, didukung, suasana, nyama...   
4  [couvee, seturan, dikenal, cabang, ramah, c

In [32]:
df['final_text'] = df['stemmed_tokens'].apply(lambda x: ' '.join(x))
print(df[['review_text', 'final_text']].head())

                                         review_text  \
0  Couvee Seturan adalah salah satu tempat nongkr...   
1  couvee dah kaya rumahh, tempatnya nyaman, fasi...   
2  Aku gak tau gimana lagi mengungkapkan rasa cin...   
3  Kopi dengan cita rasa terbaik dan didukung sua...   
4  Couvee Seturan dikenal sebagai cabang yang ram...   

                                          final_text  
0  couvee tur salah nongkrong favorit jogja suasa...  
1  couvee dah kaya rumahh tempat nyaman fasilitas...  
2  gak tau gimana cinta kopi minggu kesini nikmat...  
3  kopi cita baik dukung suasana nyaman couvee op...  
4  couvee tur kenal cabang ramah co worker ruang ...  


In [33]:
df.to_csv('cleaned_reviews.csv', index=False)

In [34]:
df

,review_text,cleaned_text,tokens,stemmed_tokens,final_text
0,Couvee Seturan adalah salah satu tempat nongkr...,couvee seturan adalah salah satu tempat nongkr...,"[couvee, seturan, salah, nongkrong, favorit, j...","[couvee, tur, salah, nongkrong, favorit, jogja...",couvee tur salah nongkrong favorit jogja suasa...
1,"couvee dah kaya rumahh, tempatnya nyaman, fasi...",couvee dah kaya rumahh tempatnya nyaman fasili...,"[couvee, dah, kaya, rumahh, tempatnya, nyaman,...","[couvee, dah, kaya, rumahh, tempat, nyaman, fa...",couvee dah kaya rumahh tempat nyaman fasilitas...
2,Aku gak tau gimana lagi mengungkapkan rasa cin...,aku gak tau gimana lagi mengungkapkan rasa cin...,"[gak, tau, gimana, cintaku, kopi, minggu, kesi...","[gak, tau, gimana, cinta, kopi, minggu, kesini...",gak tau gimana cinta kopi minggu kesini nikmat...
3,Kopi dengan cita rasa terbaik dan didukung sua...,kopi dengan cita rasa terbaik dan didukung sua...,"[kopi, cita, terbaik, didukung, suasana, nyama...","[kopi, cita, baik, dukung, suasana, nyaman, co...",kopi cita baik dukung suasana nyaman couvee op...
4,Couvee Seturan dikenal sebagai cabang yang ram...,couvee seturan dikenal sebagai cabang yang ram...,"[couvee, seturan, dikenal, cabang, ramah, co, ...","[couvee, tur, kenal, cabang, ramah, co, worker...",couvee tur kenal cabang ramah co worker ruang ...
...,...,...,...,...,...
245,"Aircon too low, kinda freezing inside. Wear wa...",aircon too low kinda freezing inside wear warm...,"[aircon, low, kinda, freezing, inside, wear, w...","[aircon, low, kinda, freezing, inside, wear, w...",aircon low kinda freezing inside wear warm clo...
246,The price lil bit pricey. Not so many choices ...,the price lil bit pricey not so many choices f...,"[price, lil, bit, pricey, many, choices, snack...","[price, lil, bit, pricey, many, choices, snack...",price lil bit pricey many choices snacksfoods ...
247,I am so in love with red velvet latte. Best in...,i am so in love with red velvet latte best in ...,"[love, red, velvet, latte, best, town]","[love, red, velvet, latte, best, town]",love red velvet latte best town
248,Thanks for excellent service 👍 barista ok ob ok,thanks for excellent service barista ok ob ok,"[thanks, excellent, service, barista, ok, ob, ok]","[thanks, excellent, service, barista, ok, ob, ok]",thanks excellent service barista ok ob ok


In [35]:
pip install transformers torch

  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 453.5 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/9.7 MB 453.5 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/9.7 MB 453.5 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/9.7 MB 453.5 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/9.7 MB 453.5 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/9.7 MB 453.5 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/9.7 MB 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 236.2 kB/s eta 0:00:06
   ------------ --------------------------- 0.5/1.7 MB 236.2 kB/s eta 0:00:06
   ------------ --------------------------- 0.5/1.7 MB 236.2 kB/s eta 0:00:06
   ----

  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------- --------------- 231.7/390.2 MB 229.1 kB/s eta 0:11:32
   ---------------------- --------------- 231.7/390.2 MB 229.1 kB/s eta 0:11:32
   ---------------------- --------------- 232.0/390.2 MB 222.3 kB/s eta 0:11:52
   ---------------------- --------------- 232.0/390.2 MB 222.3 kB/s eta 0:11:52
   ---------------------- --------------- 232.0/390.2 MB 222.3 kB/s eta 0:11:52
   ---------------------- --------------- 232.0/390.2 MB 222.3 kB/s eta 0:11:52
   ---------------------- --------------- 232.3/390.2 MB 221.3 kB/s eta 0:11:54
   ---------------------- --------------- 232.3/390.2 MB 221.3 kB/s eta 0:11:54
   ---------------------- --------------- 232.3/390.2 MB 221.3 kB/s eta 0:11:54
   ---------------------- --------------- 232.3/390.2 MB 221.3 kB/s eta 0:11:54
   ---------------------- --------------- 232.3/390.2 MB 221.3 kB/s eta 0:11:54
   ---------------------- --------------- 232.3/390.2 MB 221.3 kB/s eta 0:11:54
   ---------------------- --------------

In [41]:
from transformers import pipeline

# Inisialisasi pipeline sentimen
sentiment_pipeline = pipeline("sentiment-analysis", model="indobenchmark/indobert-base-p1")

# Fungsi untuk prediksi sentimen
def predict_sentiment(text):
    result = sentiment_pipeline(text)
    return result[0]['label']  # Mengembalikan label (positive, negative, neutral)

# Terapkan ke dataset
df['sentiment'] = df['cleaned_text'].apply(predict_sentiment)

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Cek hasil
df[['cleaned_text', 'sentiment']].head()